В файле хранятся данные об активности клиентов с сентября 2021 года по февраль 2022

Необходимо на основе этих данных провести когортный анализ.

Описание полей:

CLIENT_ID - идентификатор клиента;

VALUE_DAY - дата авторизации в мобильное приложение за последние 30 дней;


In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [4]:
df = pd.read_csv( '/dataset.csv', sep = ";")

In [5]:
df = df[['CLIENT_ID', 'VALUE_DAY']]

In [6]:
df.head()

,CLIENT_ID,VALUE_DAY
0,3596869440,30.09.2021
1,181778433,30.09.2021
2,4271531990,30.09.2021
3,3949165697,30.09.2021
4,1926920317,30.09.2021


In [ ]:
df.info()
#VALUE_DAY должен содержать тип date, а не object

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 805821 entries, 0 to 805820
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   CLIENT_ID  805821 non-null  int64 
 1   VALUE_DAY  805821 non-null  object
dtypes: int64(1), object(1)
memory usage: 12.3+ MB


In [8]:
df['VALUE_DAY'] = pd.to_datetime(df['VALUE_DAY'])

<ipython-input-8-ac06d2d7e636>:1: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['VALUE_DAY'] = pd.to_datetime(df['VALUE_DAY'])


**Находим первую дату каждого пользователя**

In [9]:
min_dt_date = df.groupby(by='CLIENT_ID', as_index=False)['VALUE_DAY'].min()

In [10]:
min_dt_date['month'] = min_dt_date['VALUE_DAY'].dt.to_period('M')

In [11]:
min_dt_date = min_dt_date[["CLIENT_ID", "month"]]

In [17]:
total_data = df.merge(min_dt_date, how='left', on = 'CLIENT_ID')

Для каждого VALUE_DAY достаем месяц

In [18]:
total_data['period'] = total_data['VALUE_DAY'].dt.to_period('M')

In [19]:
total_data

,CLIENT_ID,VALUE_DAY,month,period
0,3596869440,2021-09-30,2021-09,2021-09
1,181778433,2021-09-30,2021-09,2021-09
2,4271531990,2021-09-30,2021-09,2021-09
3,3949165697,2021-09-30,2021-09,2021-09
4,1926920317,2021-09-30,2021-09,2021-09
...,...,...,...,...
805816,4243322997,2022-02-28,2021-09,2022-02
805817,67461867,2022-02-28,2021-09,2022-02
805818,302417315,2022-02-28,2021-09,2022-02
805819,1780553365,2022-02-28,2022-01,2022-02


Находим разницу между месяцем текущей авторизации и первой авторизации клиента

In [20]:
total_data['visit_numder'] = (total_data['period'] - total_data['month']).apply(lambda x: x.n)

In [21]:
total_data

,CLIENT_ID,VALUE_DAY,month,period,visit_numder
0,3596869440,2021-09-30,2021-09,2021-09,0
1,181778433,2021-09-30,2021-09,2021-09,0
2,4271531990,2021-09-30,2021-09,2021-09,0
3,3949165697,2021-09-30,2021-09,2021-09,0
4,1926920317,2021-09-30,2021-09,2021-09,0
...,...,...,...,...,...
805816,4243322997,2022-02-28,2021-09,2022-02,5
805817,67461867,2022-02-28,2021-09,2022-02,5
805818,302417315,2022-02-28,2021-09,2022-02,5
805819,1780553365,2022-02-28,2022-01,2022-02,1


Находим количество уникальных пользователей в зависимости от того, через сколько месяцев после первой авторизации они заходили в приложение

In [22]:
cohort_data = total_data.groupby(["month", "visit_numder"], as_index=False)["CLIENT_ID"].nunique()

In [23]:
cohort_data

,month,visit_numder,CLIENT_ID
0,2021-09,0,129232
1,2021-09,1,124593
2,2021-09,2,120232
3,2021-09,3,116456
4,2021-09,4,105547
5,2021-09,5,104163
6,2021-10,0,7044
7,2021-10,1,6936
8,2021-10,2,6883
9,2021-10,3,5040


In [24]:
cohort_matrix = cohort_data.pivot(index='month', columns='visit_numder', values='CLIENT_ID')
cohort_matrix

visit_numder,0,1,2,3,4,5
month,,,,,,
2021-09,129232.0,124593.0,120232.0,116456.0,105547.0,104163.0
2021-10,7044.0,6936.0,6883.0,5040.0,4800.0,NaN
2021-11,5841.0,5743.0,5225.0,4377.0,NaN,NaN
2021-12,6875.0,6188.0,6170.0,NaN,NaN,NaN
2022-01,14770.0,14374.0,NaN,NaN,NaN,NaN
2022-02,5330.0,NaN,NaN,NaN,NaN,NaN


Достаем количество пользователей, которые авторизовывались в первый месяц после регистрации, чтобы посчитать удержание (retention)

In [25]:
size = cohort_matrix.iloc[:, 0]

In [26]:
size

,0
month,
2021-09,129232.0
2021-10,7044.0
2021-11,5841.0
2021-12,6875.0
2022-01,14770.0
2022-02,5330.0


Находим retention для каждой когорты.

In [28]:
data = cohort_matrix.divide(size, axis = 0)
data

visit_numder,0,1,2,3,4,5
month,,,,,,
2021-09,1.0,0.964103,0.930358,0.901139,0.816725,0.806016
2021-10,1.0,0.984668,0.977144,0.715503,0.681431,NaN
2021-11,1.0,0.983222,0.894539,0.749358,NaN,NaN
2021-12,1.0,0.900073,0.897455,NaN,NaN,NaN
2022-01,1.0,0.973189,NaN,NaN,NaN,NaN
2022-02,1.0,NaN,NaN,NaN,NaN,NaN


Уже можно делать выводы по удержанию пользователей в каждой когорте, но лучше визуализировать

In [31]:
data = data.reset_index()
data['month'] = data['month'].astype(str)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   month   6 non-null      object 
 1   0       6 non-null      float64
 2   1       5 non-null      float64
 3   2       4 non-null      float64
 4   3       3 non-null      float64
 5   4       2 non-null      float64
 6   5       1 non-null      float64
dtypes: float64(6), object(1)
memory usage: 468.0+ bytes


In [34]:
#делаем визуализацию
fig = px.imshow(data,
                labels=dict(x= "active users", y= "cohorts", color = 'retention'),
                )
fig.update_xaxes(side="top")
fig.show()